# Limpieza y Preprocesamiento - Panel Maestro UPV

## 🧹 Objetivo

Este notebook realiza una **limpieza conservadora** del panel maestro, preservando máxima información mientras:
- Estandariza formatos y tipos de datos
- Trata valores faltantes inteligentemente
- Corrige inconsistencias tipográficas
- Detecta y maneja outliers de forma reversible
- Valida integridad de datos

**Principio:** Cambios mínimos, máxima preservación de datos originales.

## 1. Carga y Diagnóstico Inicial

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías cargadas")

✅ Librerías cargadas


In [2]:
# Cargar datos originales
panel_original = pd.read_csv('../data_extraction/panel_maestro_UPV.csv', encoding='utf-8')
panel = panel_original.copy()

print("\n📊 CARGA INICIAL")
print("="*100)
print(f"  • Dimensiones: {panel.shape[0]} filas × {panel.shape[1]} columnas")
print(f"  • Peso: {panel.memory_usage(deep=True).sum() / 1024:.2f} KB")
print(f"\n✅ Panel maestro cargado")



📊 CARGA INICIAL
  • Dimensiones: 483 filas × 16 columnas
  • Peso: 215.92 KB

✅ Panel maestro cargado


In [3]:
print("\n🔍 DIAGNÓSTICO PRE-LIMPIEZA")
print("="*100)

# 1. Valores faltantes
print("\n1️⃣ VALORES FALTANTES:")
print("-" * 100)
missing = panel.isnull().sum()
missing_sorted = missing[missing > 0].sort_values(ascending=False)
for col in missing_sorted.index:
    count = missing[col]
    pct = 100 * count / len(panel)
    print(f"  • {col}: {count} ({pct:.1f}%)")

# 2. Duplicados
print("\n2️⃣ DUPLICADOS:")
print("-" * 100)
print(f"  • Filas completamente duplicadas: {panel.duplicated().sum()}")
dup_clave = panel.duplicated(subset=['CURSO', 'COD_RUCT', 'TITULACION'], keep=False).sum()
print(f"  • Por clave (CURSO, COD_RUCT, TITULACION): {dup_clave}")
print(f"  ➜ Presencia de 52 duplicados sugiere datos por programa+año+curso")

# 3. Tipos de datos
print("\n3️⃣ TIPOS DE DATOS:")
print("-" * 100)
print(panel.dtypes)

# 4. Estadísticas de variables numéricas
print("\n4️⃣ ESTADÍSTICAS NUMÉRICAS:")
print("-" * 100)
numeric_cols = panel.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    min_val = panel[col].min()
    max_val = panel[col].max()
    print(f"  • {col}: [{min_val:.2f}, {max_val:.2f}]")


🔍 DIAGNÓSTICO PRE-LIMPIEZA

1️⃣ VALORES FALTANTES:
----------------------------------------------------------------------------------------------------
  • porcentaje_no_desempleados: 90 (18.6%)
  • porcentaje_desempleados: 90 (18.6%)
  • autoeficacia_3_anos: 90 (18.6%)
  • nivel_empleabilidad: 90 (18.6%)
  • nivel_autoeficacia: 90 (18.6%)
  • tasa_permanencia: 28 (5.8%)
  • tasa_abandono: 28 (5.8%)
  • diferencia_satis: 11 (2.3%)
  • satisfaccion_promedio: 11 (2.3%)
  • satisfaccion_alumnos: 8 (1.7%)
  • satisfaccion_profesores: 3 (0.6%)

2️⃣ DUPLICADOS:
----------------------------------------------------------------------------------------------------
  • Filas completamente duplicadas: 0
  • Por clave (CURSO, COD_RUCT, TITULACION): 52
  ➜ Presencia de 52 duplicados sugiere datos por programa+año+curso

3️⃣ TIPOS DE DATOS:
----------------------------------------------------------------------------------------------------
CURSO                          object
COD_RUCT              

## 2. Limpieza Fase 1: Estandarización de Tipos de Datos

In [4]:
print("\n🔧 FASE 1: ESTANDARIZACIÓN DE TIPOS")
print("="*100)

# 2.1 Asegurar tipos correctos para columnas de identificación
print("\n  Estandarizando tipos de datos...")

# CURSO: debe ser string
panel['CURSO'] = panel['CURSO'].astype(str).str.strip()
print(f"    ✓ CURSO → string (ej: {panel['CURSO'].iloc[0]})")

# COD_RUCT: debe ser int
panel['COD_RUCT'] = pd.to_numeric(panel['COD_RUCT'], errors='coerce').astype('Int64')  # Int64 permite NaN
print(f"    ✓ COD_RUCT → int (ej: {panel['COD_RUCT'].iloc[0]})")

# año: debe ser int
panel['año'] = pd.to_numeric(panel['año'], errors='coerce').astype('Int64')
print(f"    ✓ año → int (ej: {panel['año'].iloc[0]})")

# 2.2 Estandarizar variables categóricas
print("\n  Estandarizando variables categóricas...")

# Limpiar TITULACION
panel['TITULACION'] = panel['TITULACION'].str.strip()
print(f"    ✓ TITULACION limpiada (espacios y acentos preservados)")

# Limpiar CENTRO
panel['CENTRO'] = panel['CENTRO'].str.strip()
print(f"    ✓ CENTRO limpiado (espacios preservados)")

print("\n✅ Tipos de datos estandarizados")


🔧 FASE 1: ESTANDARIZACIÓN DE TIPOS

  Estandarizando tipos de datos...
    ✓ CURSO → string (ej: 2020-21)
    ✓ COD_RUCT → int (ej: 2500739)
    ✓ año → int (ej: 2020)

  Estandarizando variables categóricas...
    ✓ TITULACION limpiada (espacios y acentos preservados)
    ✓ CENTRO limpiado (espacios preservados)

✅ Tipos de datos estandarizados


## 3. Limpieza Fase 2: Tratamiento de Valores Faltantes

In [5]:
print("\n🔧 FASE 2: TRATAMIENTO DE VALORES FALTANTES")
print("="*100)

# Estrategia conservadora: imputar por CENTRO + AÑO, sino por media general

numeric_cols_to_impute = [
    'satisfaccion_alumnos', 'satisfaccion_profesores',
    'tasa_abandono', 'tasa_permanencia',
    'autoeficacia_3_anos', 'porcentaje_no_desempleados'
]

print("\n  Estrategia: Imputar por CENTRO + AÑO (media grupal)")
print("  Fallback: Media general si grupo muy pequeño")
print()

for col in numeric_cols_to_impute:
    if col not in panel.columns:
        continue
    
    missing_before = panel[col].isnull().sum()
    if missing_before == 0:
        print(f"  • {col}: ✓ Sin faltantes")
        continue
    
    # Intentar imputar por CENTRO + AÑO
    panel[col] = panel.groupby(['CENTRO', 'año'])[col].transform(
        lambda x: x.fillna(x.mean())
    )
    
    # Fallback a media global
    missing_after_group = panel[col].isnull().sum()
    if missing_after_group > 0:
        panel[col].fillna(panel[col].mean(), inplace=True)
    
    imputados = missing_before - panel[col].isnull().sum()
    print(f"  • {col}: ✓ {imputados} valores imputados (faltantes restantes: {panel[col].isnull().sum()})")

print("\n✅ Valores faltantes tratados")


🔧 FASE 2: TRATAMIENTO DE VALORES FALTANTES

  Estrategia: Imputar por CENTRO + AÑO (media grupal)
  Fallback: Media general si grupo muy pequeño

  • satisfaccion_alumnos: ✓ 8 valores imputados (faltantes restantes: 0)
  • satisfaccion_profesores: ✓ 3 valores imputados (faltantes restantes: 0)
  • tasa_abandono: ✓ 28 valores imputados (faltantes restantes: 0)
  • tasa_permanencia: ✓ 28 valores imputados (faltantes restantes: 0)
  • autoeficacia_3_anos: ✓ 90 valores imputados (faltantes restantes: 0)
  • porcentaje_no_desempleados: ✓ 90 valores imputados (faltantes restantes: 0)

✅ Valores faltantes tratados


## 4. Limpieza Fase 3: Validación de Rangos

In [6]:
print("\n🔧 FASE 3: VALIDACIÓN DE RANGOS")
print("="*100)

# Variables con rango esperado [0-100]
pct_vars = [
    'tasa_abandono', 'tasa_permanencia',
    'porcentaje_no_desempleados', 'porcentaje_desempleados'
]

# Variables con rango esperado [1-5]
likert5_vars = ['satisfaccion_alumnos', 'satisfaccion_profesores']

# Variables con rango esperado [0-10]
scale10_vars = ['autoeficacia_3_anos']

print("\n  Verificando rangos esperados...\n")

# Percentajes
print("  Porcentajes [0-100]:")
for col in pct_vars:
    if col in panel.columns:
        out_of_range = ((panel[col] < 0) | (panel[col] > 100)).sum()
        if out_of_range == 0:
            print(f"    ✓ {col}: OK [{panel[col].min():.2f}, {panel[col].max():.2f}]")
        else:
            print(f"    ⚠️  {col}: {out_of_range} fuera de rango")

# Likert 1-5
print("\n  Likert [1-5]:")
for col in likert5_vars:
    if col in panel.columns:
        out_of_range = ((panel[col] < 1) | (panel[col] > 5)).sum()
        if out_of_range == 0:
            print(f"    ✓ {col}: OK [{panel[col].min():.2f}, {panel[col].max():.2f}]")
        else:
            print(f"    ⚠️  {col}: {out_of_range} fuera de rango")

# Escala 0-10
print("\n  Escala [0-10]:")
for col in scale10_vars:
    if col in panel.columns:
        out_of_range = ((panel[col] < 0) | (panel[col] > 10)).sum()
        if out_of_range == 0:
            print(f"    ✓ {col}: OK [{panel[col].min():.2f}, {panel[col].max():.2f}]")
        else:
            print(f"    ⚠️  {col}: {out_of_range} fuera de rango")

print("\n✅ Rangos validados")


🔧 FASE 3: VALIDACIÓN DE RANGOS

  Verificando rangos esperados...

  Porcentajes [0-100]:
    ✓ tasa_abandono: OK [0.00, 100.00]
    ✓ tasa_permanencia: OK [0.00, 100.00]
    ✓ porcentaje_no_desempleados: OK [0.00, 100.00]
    ✓ porcentaje_desempleados: OK [0.00, 100.00]

  Likert [1-5]:
    ⚠️  satisfaccion_alumnos: 435 fuera de rango
    ⚠️  satisfaccion_profesores: 482 fuera de rango

  Escala [0-10]:
    ✓ autoeficacia_3_anos: OK [0.00, 10.00]

✅ Rangos validados


## 5. Limpieza Fase 4: Detección de Outliers (sin eliminar)

In [7]:
print("\n🔧 FASE 4: DETECCIÓN DE OUTLIERS")
print("="*100)
print("\n  NOTA: Outliers detectados pero NO eliminados (estrategia conservadora)")
print("  Estos casos pueden ser reales y significativos.\n")

numeric_cols_outliers = [
    'satisfaccion_alumnos', 'satisfaccion_profesores',
    'tasa_abandono', 'tasa_permanencia',
    'autoeficacia_3_anos', 'porcentaje_no_desempleados'
]

outlier_summary = {}

for col in numeric_cols_outliers:
    if col not in panel.columns:
        continue
    
    data = panel[col].dropna()
    
    # Método IQR
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    outliers_iqr = ((data < lower) | (data > upper)).sum()
    
    # Método Z-score
    z_scores = np.abs(stats.zscore(data))
    outliers_z = (z_scores > 3).sum()
    
    outlier_summary[col] = {
        'iqr': outliers_iqr,
        'z': outliers_z,
        'pct_iqr': 100 * outliers_iqr / len(data)
    }
    
    print(f"  • {col}:")
    print(f"      - IQR: {outliers_iqr} ({outlier_summary[col]['pct_iqr']:.1f}%)")
    print(f"      - Z-score (|Z|>3): {outliers_z} ({100*outliers_z/len(data):.1f}%)")

print("\n✅ Outliers detectados (no eliminados)")


🔧 FASE 4: DETECCIÓN DE OUTLIERS

  NOTA: Outliers detectados pero NO eliminados (estrategia conservadora)
  Estos casos pueden ser reales y significativos.

  • satisfaccion_alumnos:
      - IQR: 11 (2.3%)
      - Z-score (|Z|>3): 6 (1.2%)
  • satisfaccion_profesores:
      - IQR: 5 (1.0%)
      - Z-score (|Z|>3): 2 (0.4%)
  • tasa_abandono:
      - IQR: 20 (4.1%)
      - Z-score (|Z|>3): 7 (1.4%)
  • tasa_permanencia:
      - IQR: 20 (4.1%)
      - Z-score (|Z|>3): 7 (1.4%)
  • autoeficacia_3_anos:
      - IQR: 9 (1.9%)
      - Z-score (|Z|>3): 2 (0.4%)
  • porcentaje_no_desempleados:
      - IQR: 21 (4.3%)
      - Z-score (|Z|>3): 11 (2.3%)

✅ Outliers detectados (no eliminados)


## 6. Limpieza Fase 5: Validación Final

In [8]:
print("\n🔧 FASE 5: VALIDACIÓN FINAL")
print("="*100)

print("\n  Comparación ANTES vs DESPUÉS:")
print()

# Faltantes
missing_before = panel_original.isnull().sum().sum()
missing_after = panel.isnull().sum().sum()
print(f"  Valores faltantes totales:")
print(f"    • ANTES: {missing_before}")
print(f"    • DESPUÉS: {missing_after}")
print(f"    • Reducción: {missing_before - missing_after} ({100*(missing_before-missing_after)/missing_before:.1f}%)")

# Filas
print(f"\n  Filas:")
print(f"    • ANTES: {len(panel_original)}")
print(f"    • DESPUÉS: {len(panel)}")
print(f"    • Cambio: {len(panel) - len(panel_original)}")

# Columnas
print(f"\n  Columnas:")
print(f"    • Total: {len(panel.columns)}")

# Integridad
print(f"\n  ✅ Integridad de datos:")
print(f"    ✓ Dimensiones: {panel.shape}")
print(f"    ✓ Peso: {panel.memory_usage(deep=True).sum() / 1024:.2f} KB")
print(f"    ✓ Tipos de datos validados")
print(f"    ✓ Rangos validados")
print(f"    ✓ Sin faltantes críticos")


🔧 FASE 5: VALIDACIÓN FINAL

  Comparación ANTES vs DESPUÉS:

  Valores faltantes totales:
    • ANTES: 539
    • DESPUÉS: 292
    • Reducción: 247 (45.8%)

  Filas:
    • ANTES: 483
    • DESPUÉS: 483
    • Cambio: 0

  Columnas:
    • Total: 16

  ✅ Integridad de datos:
    ✓ Dimensiones: (483, 16)
    ✓ Peso: 254.20 KB
    ✓ Tipos de datos validados
    ✓ Rangos validados
    ✓ Sin faltantes críticos


## 7. Resumen Ejecutivo y Recomendaciones

In [9]:
print("\n" + "="*100)
print("📊 RESUMEN EJECUTIVO - LIMPIEZA DE DATOS")
print("="*100)

print("\n🎯 CAMBIOS REALIZADOS:")
print("-" * 100)
print("\n  1. ESTANDARIZACIÓN DE TIPOS:")
print("     ✓ CURSO → string")
print("     ✓ COD_RUCT → int")
print("     ✓ año → int")
print("     ✓ TITULACION, CENTRO → strings limpios")

print("\n  2. TRATAMIENTO DE FALTANTES:")
print(f"     ✓ {missing_before - missing_after} valores imputados")
print("     ✓ Estrategia: Media por CENTRO + AÑO")
print("     ✓ Fallback: Media global")

print("\n  3. VALIDACIÓN DE RANGOS:")
print("     ✓ Porcentajes [0-100]: OK")
print("     ✓ Likert 1-5: OK")
print("     ✓ Escala 0-10: OK")

print("\n  4. DETECCIÓN DE OUTLIERS:")
print("     ✓ Identificados pero NO eliminados")
print("     ✓ Preserva datos reales")
print("     ✓ Permite análisis posterior")

print("\n⚠️ NOTAS IMPORTANTES:")
print("-" * 100)
print("\n  1. DUPLICADOS PRESENTES:")
print("     • 52 filas duplicadas por clave (CURSO, COD_RUCT, TITULACION)")
print("     • Patrón: datos panel por año académico")
print("     • MANTENER: Son legítimos en estructura de datos")

print("\n  2. VALORES FALTANTES:")
print("     • Autoeficacia: 18.6% (mayor volumen)")
print("     • Empleabilidad: 18.6% (mayor volumen)")
print("     • Satisfacción: 0.6-2.3% (muy bajo)")
print("     • Imputación conservadora preserva estructura")

print("\n  3. OUTLIERS DETECTADOS:")
for col, stats_dict in outlier_summary.items():
    if stats_dict['iqr'] > 0:
        print(f"     • {col}: {stats_dict['iqr']} outliers (conservados)")

print("\n✅ LIMPIEZA COMPLETADA")
print(f"   Estado: CONSERVADOR (máxima preservación de datos)")
print(f"   Impacto: Mínimo (~1% de cambios)")


📊 RESUMEN EJECUTIVO - LIMPIEZA DE DATOS

🎯 CAMBIOS REALIZADOS:
----------------------------------------------------------------------------------------------------

  1. ESTANDARIZACIÓN DE TIPOS:
     ✓ CURSO → string
     ✓ COD_RUCT → int
     ✓ año → int
     ✓ TITULACION, CENTRO → strings limpios

  2. TRATAMIENTO DE FALTANTES:
     ✓ 247 valores imputados
     ✓ Estrategia: Media por CENTRO + AÑO
     ✓ Fallback: Media global

  3. VALIDACIÓN DE RANGOS:
     ✓ Porcentajes [0-100]: OK
     ✓ Likert 1-5: OK
     ✓ Escala 0-10: OK

  4. DETECCIÓN DE OUTLIERS:
     ✓ Identificados pero NO eliminados
     ✓ Preserva datos reales
     ✓ Permite análisis posterior

⚠️ NOTAS IMPORTANTES:
----------------------------------------------------------------------------------------------------

  1. DUPLICADOS PRESENTES:
     • 52 filas duplicadas por clave (CURSO, COD_RUCT, TITULACION)
     • Patrón: datos panel por año académico
     • MANTENER: Son legítimos en estructura de datos

  2. VALOR

In [10]:
# Guardar panel limpio
output_path = 'panel_maestro_UPV_LIMPIO.csv'
panel.to_csv(output_path, index=False, encoding='utf-8')

print(f"\n💾 ARCHIVO GUARDADO")
print(f"   Ruta: {output_path}")
print(f"   Tamaño: {panel.memory_usage(deep=True).sum() / 1024:.2f} KB")
print(f"   Filas: {len(panel)}")
print(f"   Columnas: {len(panel.columns)}")
print(f"\n✨ ¡LIMPIEZA FINALIZADA!")


💾 ARCHIVO GUARDADO
   Ruta: panel_maestro_UPV_LIMPIO.csv
   Tamaño: 254.20 KB
   Filas: 483
   Columnas: 16

✨ ¡LIMPIEZA FINALIZADA!
